In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import nibabel as nib

from scipy import ndimage


def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [2]:
# creating the paths for each respective class folder 
non_target_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/0", x)
    for x in os.listdir("labeled_data/data/0")
]


saline_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/1", x)
    for x in os.listdir("labeled_data/data/1")
]

rubber_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/2", x)
    for x in os.listdir("labeled_data/data/2")
]

clay_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/3", x)
    for x in os.listdir("labeled_data/data/3")
]

print("scans with non target: " + str(len(non_target_paths)))
print("scans with saline: " + str(len(saline_paths)))
print("scans with rubber: " + str(len(rubber_paths)))
print("scans with clay: " + str(len(clay_paths)))

scans with non target: 707
scans with saline: 106
scans with rubber: 112
scans with clay: 80


In [3]:
from sklearn.utils import resample
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
non_target_scans = np.array([process_scan(path) for path in non_target_paths])
saline_scans = np.array([process_scan(path) for path in saline_paths])
rubber_scans = np.array([process_scan(path) for path in rubber_paths])
clay_scans = np.array([process_scan(path) for path in clay_paths])

# max_non = np.max([image.shape for image in non_target_scans], axis=0)
# max_saline = np.max([image.shape for image in saline_scans], axis=0)
# max_rubber = np.max([image.shape for image in rubber_scans], axis=0)
# max_clay = np.max([image.shape for image in clay_scans], axis=0)


# print(max_non)
# print(max_saline)
# print(max_rubber)
# print(max_clay)

# assign 1 for target, for the non_target ones assign 0.
non_target_labels = np.array([0 for _ in range(len(non_target_scans))])
saline_labels = np.array([1 for _ in range(len(saline_scans))])
rubber_labels = np.array([1 for _ in range(len(rubber_scans))])
clay_labels = np.array([1 for _ in range(len(clay_scans))])


def generate_rotations(nparray, labels):
    for img in nparray:
        for i in range(2):
            new_img = np.rot90(img)
            nparray = np.append(nparray, np.expand_dims(new_img, axis=0), axis=0)
            labels = np.append(labels, labels[-1])
    return nparray, labels

saline_scans, saline_labels = generate_rotations(saline_scans, saline_labels)
rubber_scans, rubber_labels = generate_rotations(rubber_scans, rubber_labels)
clay_scans, clay_labels = generate_rotations(clay_scans, clay_labels)

# print("scans with saline: " + str(len(saline_scans)))
# print("scans with rubber: " + str(len(rubber_scans)))
# print("scans with clay: " + str(len(clay_scans)))


x_train = np.concatenate((non_target_scans[:round(len(non_target_scans)*0.8)], saline_scans[:round(len(saline_scans)*0.8)], rubber_scans[:round(len(rubber_scans)*0.8)], clay_scans[:round(len(clay_scans)*0.8)]), axis=0)
y_train = np.concatenate((non_target_labels[:round(len(non_target_labels)*0.8)], saline_labels[:round(len(saline_labels)*0.8)], rubber_labels[:round(len(rubber_labels)*0.8)], clay_labels[:round(len(clay_labels)*0.8)]), axis=0)
x_test = np.concatenate((non_target_scans[round(len(non_target_scans)*0.8):], saline_scans[round(len(saline_scans)*0.8):], rubber_scans[round(len(rubber_scans)*0.8):], clay_scans[round(len(clay_scans)*0.8):]), axis=0)
y_test = np.concatenate((non_target_labels[round(len(non_target_labels)*0.8):], saline_labels[round(len(saline_labels)*0.8):], rubber_labels[round(len(rubber_labels)*0.8):], clay_labels[round(len(clay_labels)*0.8):]), axis=0)
print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_test.shape[0])
)

y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

print(y_train)


# # Undersample the non-targets class (WAS NOT ACCURATE DID NOT WORK)

# n_minority = min(len(saline_scans), len(rubber_scans), len(clay_scans))
# non_target_scans_undersampled, non_target_labels_undersampled = resample(non_target_scans,
#                                                                          non_target_labels,
#                                                                          replace=False,  # Sample without replacement
#                                                                          n_samples=400,  # Number of samples to match minority class
#                                                                          )  # Set random seed for reproducibility

# # Combine the minority classes and the undersampled non-targets class
# x_train = np.concatenate((non_target_scans_undersampled, saline_scans, rubber_scans, clay_scans), axis=0)
# y_train = np.concatenate((non_target_labels_undersampled, saline_labels, rubber_labels, clay_labels), axis=0)
# x_val = np.concatenate((non_target_scans[round(len(non_target_scans)*0.8):], saline_scans[round(len(saline_scans)*0.8):], rubber_scans[round(len(rubber_scans)*0.8):], clay_scans[round(len(clay_scans)*0.8):]), axis=0)
# y_val = np.concatenate((non_target_labels[round(len(non_target_labels)*0.8):], saline_labels[round(len(saline_labels)*0.8):], rubber_labels[round(len(rubber_labels)*0.8):], clay_labels[round(len(clay_labels)*0.8):]), axis=0)
# print(
#     "Number of samples in train and validation are %d and %d."
#     % (x_train.shape[0], x_val.shape[0])
# )

# Split data in the ratio 80-20 for training and validation.


Number of samples in train and validation are 1281 and 320.
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [4]:
import random

from scipy import ndimage

def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


In [5]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(x_train,y_train, test_size=0.2) # remove when done with gridsearch

# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((train_x, train_y))
validation_loader = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

batch_size = 16
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(train_x))
    .map(train_preprocessing)
    .batch(batch_size)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(valid_x))
    .map(validation_preprocessing)
    .batch(batch_size)
)


In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# recreation of CNN architecture to perform gridsearchCV to tune hyper parameters 

# SKIP

def create_model(filters, kernel_size, dense_units, dropout):
    inputs = keras.Input((128, 128, 64, 1))
    x = layers.Conv3D(filters=filters, kernel_size=kernel_size, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv3D(filters=filters, kernel_size=kernel_size, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv3D(filters=2*filters, kernel_size=kernel_size, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv3D(filters=4*filters, kernel_size=kernel_size, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=dense_units, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(units=2, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(),
        metrics=["accuracy"],
    )
    return model

# Define hyperparameters to search
param_grid = {
    'filters': [32, 64],
    'kernel_size': [3, 5],
    'dense_units': [128, 256],
    'dropout': [0.3, 0.5]
}



# Create KerasClassifier for grid search
model = KerasClassifier(build_fn=create_model, batch_size=2, verbose=1)

# Run grid search with cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x,train_y, validation_data=(valid_x,valid_y))

# Print best results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [7]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    # gridsearchCV, 1 ---------------------------------------------------------------------------------------

    # inputs = keras.Input((width, height, depth, 1))


    # x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(inputs)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.GlobalAveragePooling3D()(x)
    # x = layers.Dense(units=256, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)

    # outputs = layers.Dense(units=2, activation="sigmoid")(x)


    # # Define the model.
    # model = keras.Model(inputs, outputs, name="3dcnn")
    # return model

    #gridsearchCV, {'dense_units': 128, 'dropout': 0.5, 'filters': 64, 'kernel_size': 3} , removed the rotation from the preprocessing pipeline
    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(units=2, activation="sigmoid")(x)


    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


    #inital architecture no tuning ------------------------------------------------------------------------------------

    # inputs = keras.Input((width, height, depth, 1))

    # x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=2)(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.GlobalAveragePooling3D()(x)
    # x = layers.Dense(units=512, activation="relu")(x)
    # x = layers.Dropout(0.3)(x)

    # outputs = layers.Dense(units=2, activation="sigmoid")(x)

    # # Define the model.
    # model = keras.Model(inputs, outputs, name="3dcnn")
    # return model


model = get_model(width=128, height=128, depth=64)
model.summary()


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 64, 1)  0         
                             ]                                   
                                                                 
 conv3d_4 (Conv3D)           (None, 126, 126, 62, 64)  1792      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 63, 63, 31, 64)   0         
 3D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 63, 63, 31, 64)   256       
 hNormalization)                                                 
                                                                 
 conv3d_5 (Conv3D)           (None, 61, 61, 29, 64)    110656    
                                                             

In [8]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


[]
Num GPUs Available:  0


In [9]:
# print(d_class_weights)
import matplotlib.pyplot as plt


# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=100,
    shuffle=True,
    verbose=1,
    callbacks=[checkpoint_cb, early_stopping_cb],
    # class_weight = d_class_weights
)


# Plot accuracy vs validation accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


#  loss: 0.9537 - acc: 0.7065 - val_loss: 0.9253 - val_acc: 0.6894

Epoch 1/100
 3/64 [>.............................] - ETA: 13:29 - loss: 0.6375 - acc: 0.6042

In [ ]:
from sklearn.metrics import accuracy_score

model.load_weights("3d_image_classification.h5")
y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
acc = accuracy_score(y_test_labels, y_pred_labels)
print("Accuracy:", acc)

In [ ]:
import pandas as pd

unlabeled_data_paths = [
    os.path.join(os.getcwd(), "unlabeled_data/", x)
    for x in os.listdir("unlabeled_data/")
]

print("scans with unlabeled data: " + str(len(unlabeled_data_paths)))

unlabeled_data_scans = np.array([process_scan(path) for path in unlabeled_data_paths])


gt_y = model.predict(unlabeled_data_scans)

In [ ]:
print(gt_y.shape)
print(type(gt_y))
class_labels = ["0", "1"] # replace with your own class labels
predicted_labels = [class_labels[prediction.argmax()] for prediction in gt_y]

print(predicted_labels)

class_counts = np.bincount(predicted_labels) # count the number of occurrences of each predicted class
print(class_counts)

In [ ]:
ground_truth = pd.read_excel('testing_filenames.xlsx', names=['filename'], index_col=None, header=None)
for i in ground_truth.index:
    filename = ground_truth['filename'].iloc[i]

df = pd.DataFrame({'filename': ground_truth['filename'], 'label': predicted_labels})
df = df.set_index('filename')
df = df.loc[ground_truth['filename']]
df.to_excel('y_predictions_new1.xlsx')
